In [1]:
import time
import numpy as np
import pandas as pd

In [2]:
import keras
from keras.engine.topology import Layer

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [3]:
def mape_loss_func(preds, labels):
    mask = labels > 5
    return np.mean(np.fabs(labels[mask]-preds[mask])/labels[mask])

def smape_loss_func(preds, labels):
    mask= labels > 5
    return np.mean(2*np.fabs(labels[mask]-preds[mask])/(np.fabs(labels[mask])+np.fabs(preds[mask])))

def mae_loss_func(preds, labels):
    mask= labels > 5
    return np.fabs((labels[mask]-preds[mask])).mean()

def eliminate_nan(b):
    a = np.array(b)
    c = a[~np.isnan(a)]
    return c

In [4]:
randseed = 25
res = 11

v = pd.read_csv('../data/v_20_aggragated.csv')
v = v.rename(columns={'Unnamed: 0': 'id'})
det_with_class = pd.read_csv('../res/%i_res%i_id_402_withclass.csv'%(randseed, res), index_col=0)

v['class_i'] = ''
for i in range(len(v)):
    v.loc[i, 'class_i'] = det_with_class[det_with_class['id']==v.loc[i, 'id']].iloc[0, 5]  # 5 stands for 'class_i'

num_class = det_with_class['class_i'].drop_duplicates().size

v_class = []
for i in range(num_class):
    v_class.append(v[v['class_i']==i])

print('There are %i class(es)'%num_class)

dist_mat = pd.read_csv('../data/dist_mat.csv', index_col=0)
id_info = pd.read_csv('../data/id2000.csv', index_col=0)
dist_mat.index = id_info['id2']
dist_mat.columns = id_info['id2']
for i in range(len(dist_mat)):
    for j in range(len(dist_mat)):
        if i==j:
            dist_mat.iloc[i, j] = 0

near_id = pd.DataFrame(np.argsort(np.array(dist_mat)), index = id_info['id2'], columns = id_info['id2'])

def get_node(det, seg):
    # det is one single detector id
    # node is one single node id
    
    # seg = pd.read_csv('./data/segement.csv', header=None)
    try:
        node_info = seg[seg[6]==det]
        node = node_info.iloc[0, 0]
    except:
        node_info = seg[seg[7]==det]
        node = node_info.iloc[0, 0]
        
    return node

def get_class_with_node(seg, v_class):
    det_list_class = np.array([])
    try:
        v_class.insert(1, 'id2', '')  # id2 mean node id
    except:
        v_class['id2'] = ''
        
    for i in range(len(v_class)):
        det_list_class = np.append(det_list_class, v_class.iloc[i, 0])
        v_class.iloc[i, 1] = get_node(v_class.iloc[i, 0], seg)
    
    return det_list_class, v_class

def rds_mat(old_dist_mat, det_ids):
    # get a matrix that contains n raods that have specified node id s
    node_ids = np.array([])
    for i in det_ids:
        node_ids = np.append(node_ids, get_node(i, seg))
        
    new_dist_mat = old_dist_mat.loc[node_ids, node_ids]
    old_dist_mat = np.array(old_dist_mat)
    new_near_id_mat = np.argsort(new_dist_mat)
    return new_near_id_mat

seg = pd.read_csv('../data/segement.csv', header=None)
num_dets = 25

det_list_class = []
for i in range(num_class):
    det_list_class_temp, v_class_temp = get_class_with_node(seg, v_class[i])
    det_list_class.append(det_list_class_temp)
    v_class_temp = v_class_temp[v_class_temp['id'].isin(det_list_class_temp[:num_dets])]
    v_class[i] = v_class_temp
    
near_road_set = []
for i in range(num_class):
    near_road_set.append(rds_mat(dist_mat, det_list_class[i][:num_dets]))

There are 5 class(es)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [5]:
def sliding_window(flow, near_road, from_day, to_day, prop):
    flow = np.array(flow)
    # 选数据
    flow = flow[:, 144*(from_day-1):144*(to_day)]
    print(flow.shape)

    # 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式

    global k, t_p, t_input, t_pre, num_links
    k = 5 # 参数k为需考虑的最近路段数
    t_p = 10 # 参数t_p为总时间序列长度（天）
    t_input = 12 #参数t_input为输入时间窗(10min颗粒度)
    t_pre = 3 #参数t_pre为预测时间窗(10min颗粒度)
    num_links = 25 #参数num_links为总路段数


    image = []
    for i in range(np.shape(near_road)[0]):
        road_id = []
        for j in range(k):
            road_id.append(near_road[i][j])
        image.append(flow[road_id, :])
    image1 = np.reshape(image, [-1, k, len(flow[0,:])])
    image2 = np.transpose(image1,(1,2,0))
    image3 = []
    label = []
    day = []

    for i in range(1, t_p):
        for j in range(144-t_input-t_pre):
            image3.append(image2[:, i*144+j:i*144+j+t_input, :][:])
            label.append(flow[:, i*144+j+t_input:i*144+j+t_input+t_pre][:])
            day.append(flow[:, (i-1)*144+j+t_input:(i-1)*144+j+t_input+t_pre][:])

    image3 = np.asarray(image3)
    label = np.asarray(label)
    day =  np.asarray(day)

    print(np.shape(image3))

    #划分前90%数据为训练集，最后10%数据为测试集
    image_train = image3[:int(np.shape(image3)[0]*prop)]
    image_test = image3[int(np.shape(image3)[0]*prop):]
    label_train = label[:int(np.shape(label)[0]*prop)]
    label_test = label[int(np.shape(label)[0]*prop):]

    day_train = day[:int(np.shape(day)[0]*prop)]
    day_test = day[int(np.shape(day)[0]*prop):]

    print(image_train.shape)
    print(image_test.shape)
    print(label_train.shape)
    print(label_test.shape)
    
    return image_train, image_test, day_train, day_test, label_train, label_test

In [6]:
# ind, class
# 0  , blue
# 1  , green
# 2  , yellow  <--
# 3  , black   <--
# 4  , red     <--
class_color_set = ['b', 'g', 'y', 'black', 'r']
class_i = 2

near_road = np.array(near_road_set[class_i])
flow = v_class[class_i].iloc[:, 2:-1]

prop = 24/31  # proportion of training data

image_train, image_test, day_train, day_test, label_train, label_test\
= sliding_window(
    flow, near_road, from_day=1, to_day=31, prop=prop
)

(25, 4464)
(1161, 5, 12, 25)
(898, 5, 12, 25)
(263, 5, 12, 25)
(898, 25, 3)
(263, 25, 3)


In [7]:
# define merge layer
class Merge_Layer(Layer):
    def __init__(self, **kwargs):
        super(Merge_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.para1 = self.add_weight(shape=(input_shape[0][1], input_shape[0][2]),
                                     initializer='uniform', trainable=True,
                                     name='para1')
        self.para2 = self.add_weight(shape=(input_shape[1][1], input_shape[1][2]),
                                     initializer='uniform', trainable=True,
                                     name='para2')
        super(Merge_Layer, self).build(input_shape)

    def call(self, inputs):
        mat1 = inputs[0]
        mat2 = inputs[1]
        output = mat1 * self.para1 + mat2 * self.para2
        # output = mat1 * 0.1 + mat2 * 0.9
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [8]:
input_data = keras.Input(shape=(k,t_input,num_links), name='input_data')
input_HA = keras.Input(shape=(num_links, t_pre), name='input_HA')

x = keras.layers.BatchNormalization(input_shape =(k,t_input,num_links))(input_data)

x = keras.layers.Conv2D(
                           filters = num_links,
                           kernel_size = 3,
                           strides = 1,
                           padding="SAME",
                           activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)

x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(
                       filters = num_links,
                       kernel_size = 3,
                       strides = 1,
                       padding="SAME",
                       activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)

x = keras.layers.Dense(num_links*2*t_pre, activation='relu', name='dense_1')(x)
x = keras.layers.Dense(num_links*t_pre, activation='relu', name='dense_2')(x)

output = keras.layers.Reshape((num_links,t_pre))(x)

output_final = Merge_Layer()([output, input_HA])

# construct model
finish_model = keras.models.Model([input_data,input_HA], [output_final])

finish_model.summary()








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 5, 12, 25)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 5, 12, 25)    100         input_data[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 5, 12, 25)    5650        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 5, 12, 25)    0           conv2d_1[0][0]       

In [9]:
finish_model.compile(optimizer='adam', loss='mean_squared_error')

In [10]:
X_train = image_train
X_HA_train = day_train
label_train = label_train

In [11]:
#模型拟合与评估
finish_model.fit([X_train,X_HA_train], label_train, epochs=100, batch_size=128,
validation_data=([image_test,day_test], label_test))
# finish_model.evaluate(image_test, label_test)

Train on 898 samples, validate on 263 samples
Epoch 1/100
898/898 [==============================] - 1s 1ms/step - loss: 4057.3298 - val_loss: 3901.6988
Epoch 2/100
898/898 [==============================] - 0s 86us/step - loss: 3978.8333 - val_loss: 3784.0184
Epoch 3/100
898/898 [==============================] - 0s 88us/step - loss: 3871.0251 - val_loss: 3602.8735
Epoch 4/100
898/898 [==============================] - 0s 92us/step - loss: 3704.2863 - val_loss: 3329.2337
Epoch 5/100
898/898 [==============================] - 0s 89us/step - loss: 3436.2273 - val_loss: 2941.6293
Epoch 6/100
898/898 [==============================] - 0s 87us/step - loss: 3047.9091 - val_loss: 2506.9249
Epoch 7/100
898/898 [==============================] - 0s 89us/step - loss: 2580.8703 - val_loss: 2229.8790
Epoch 8/100
898/898 [==============================] - 0s 81us/step - loss: 2181.8227 - val_loss: 2205.3202
Epoch 9/100
898/898 [==============================] - 0s 82us/step - loss: 1992.9389 - val

898/898 [==============================] - 0s 80us/step - loss: 408.6724 - val_loss: 389.9944
Epoch 77/100
898/898 [==============================] - 0s 79us/step - loss: 401.2798 - val_loss: 378.8777
Epoch 78/100
898/898 [==============================] - ETA: 0s - loss: 394.421 - 0s 76us/step - loss: 394.8323 - val_loss: 364.6513
Epoch 79/100
898/898 [==============================] - 0s 82us/step - loss: 387.0577 - val_loss: 355.2471
Epoch 80/100
898/898 [==============================] - 0s 83us/step - loss: 380.4459 - val_loss: 349.0973
Epoch 81/100
898/898 [==============================] - 0s 80us/step - loss: 373.0723 - val_loss: 342.7726
Epoch 82/100
898/898 [==============================] - 0s 77us/step - loss: 362.7571 - val_loss: 336.3631
Epoch 83/100
898/898 [==============================] - 0s 79us/step - loss: 356.2508 - val_loss: 328.5857
Epoch 84/100
898/898 [==============================] - 0s 78us/step - loss: 349.3915 - val_loss: 322.1546
Epoch 85/100
898/898 [==

In [12]:
#模型预测
model_pre = finish_model.predict([image_test,day_test])

In [13]:
#计算各项误差指标

mape_mean = mape_loss_func(model_pre, label_test)
smape_mean = smape_loss_func(model_pre, label_test)
mae_mean = mae_loss_func(model_pre, label_test)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.23551441632811373
smape = 0.2438822545933385
mae = 11.871502050023368


In [14]:
#模型保存
finish_model.save_weights('../model/source_%s.h5'%class_color_set[class_i])


In [15]:
#计算每条路段的误差
mape_list = []
for i in range(num_links):
    a1 = mape_loss_func(model_pre[:,i,:], label_test[:,i,:])
    mape_list.append(a1)
    print(str(i+1)+'th link')
    print(a1)

1th link
0.26261731765667157
2th link
0.39090020482780297
3th link
0.1534360003968672
4th link
0.07588254645986028
5th link
0.19515292140225413
6th link
0.1647863351166111
7th link
0.30753262023317013
8th link
0.3324902440080027
9th link
0.17535663543600474
10th link
0.2605980091906449
11th link
0.2677752044830093
12th link
0.26809432196457095
13th link
0.21822644966872412
14th link
0.17335147764714917
15th link
0.36128027921638106
16th link
0.2037353469657516
17th link
0.21665970945002286
18th link
0.09790202182768269
19th link
0.3991950659697375
20th link
0.14337139686771277
21th link
0.2969770487845934
22th link
0.18366100038070074
23th link
0.3532441137242602
24th link
0.11038035088579141
25th link
0.27525378563886527


In [16]:
mape_pd = pd.Series(mape_list)

In [17]:
mape_pd.sort_values()

3     0.075883
17    0.097902
23    0.110380
19    0.143371
2     0.153436
5     0.164786
13    0.173351
8     0.175357
21    0.183661
4     0.195153
15    0.203735
16    0.216660
12    0.218226
9     0.260598
0     0.262617
10    0.267775
11    0.268094
24    0.275254
20    0.296977
6     0.307533
7     0.332490
22    0.353244
14    0.361280
1     0.390900
18    0.399195
dtype: float64